<a href="https://colab.research.google.com/github/Tannongma/SCM.275x/blob/main/SCM_275x_Inventory_Pooling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SCM.275x - Advanced Supply Chain Systems Planning and Network Design
# **Inventory Pooling**

### *Before starting, make sure to save a copy of this notebook to your Google Drive!*

## **Initialization**

In [ ]:
# Install necessary packages if they are not already installed

!pip install gurobipy   # Gurobi optimization solver
!pip install pandas     # Pandas for data analysis and manipulation
!pip install folium     # Folium for creating interactive maps
!pip install geopy      # Geopy for computing distances and working with geographic data


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 61.8 MB/s eta 0:00:00


In [ ]:
# Import all required packages

import pandas as pd                   # For data manipulation and analysis
import gurobipy as grb                # Gurobi optimization library for solving mathematical models
import folium                         # For creating interactive maps
import folium.plugins as plugins      # Additional plugins for folium
from geopy.distance import geodesic   # For calculating geodesic distances between two points
import scipy.stats as st              # Importing the stats module from SciPy for statistical functions (e.g., probability distributions)
import math                           # Importing the math module for mathematical functions (e.g., logarithms, square roots)


## **Helper functions**

### **Ploting nodes on a map**

In [ ]:
# Defining a function to plot nodes on a map using folium

def plot_nodes(map,                         # Folium map object to plot the nodes on
               nodes,                       # Dictionary of node objects where each node contains attributes like latitude and longitude
               icon,                        # Icon symbol to use for the markers on the map
               active_color,                # Color of the marker icon for active nodes
               background_color,            # Background color of the marker icon
               inactive_color = 'grey',     # Color of the marker icon for inactive nodes
               ):

    # Loop through each node in the dictionary
    for node in nodes.values():

        # Create a folium marker
        marker = folium.Marker(
            location=[node.lat, node.lon],              # Set the marker's location
            popup = (node.ID + "-" + node.name),        # Create a marker popup with the node ID and name
            icon=plugins.BeautifyIcon(                  # Create a marker's icon
                icon=icon,
                icon_shape="circle",
                text_color=active_color if node.active == True else inactive_color,
                border_color=active_color if node.active == True else inactive_color,
                background_color=background_color,
            )
        )

        # Add a folium marker to the map
        marker.add_to(map)


### **Computing geodesic distance**

In [ ]:
# Defining a function for computing geodesic distances between two locations

def compute_geodesic_distance(origin,       # Origin node object
                              destination,  # Destination node
                              unit='km'):   # Unit ('km' or 'mi')

    # Extract coordinates (latitude and longitude) from origin and destination
    origin_coordinates = [origin.lat, origin.lon]
    destination_coordinates = [destination.lat, destination.lon]

    # Compute distance based on the specified unit
    if unit == 'km':
        distance = geodesic(origin_coordinates, destination_coordinates).km  # Compute distance in kilometers
    elif unit == 'mi':
        distance = geodesic(origin_coordinates, destination_coordinates).mi  # Compute distance in miles

    return distance  # Return the calculated distance


### **Ploting flows on a map**

In [ ]:
# Defining a function to plot flows on a map using folium

def plot_flows(map,                   # Folium map object where flows will be plotted.
               vars,                  # Dictionary of decision variables from the optimization model
               nodes,                 # Dictionary of node objects
               width = 10,            # Line width
               color = 'grey',        # Color of the lines representing flows, default is grey
               opacity = 0.5):        # Opacity of the lines, default is 0.5

    # Iterate over flow decision variables (keys represent node pairs)
    for (node1_key, node2_key), var in vars.items():

        # Plot only positive flows
        if var.X > 0:

            # Get the coordinates of the nodes for plotting the line
            points = [[nodes[node1_key].lat, nodes[node1_key].lon],
                      [nodes[node2_key].lat, nodes[node2_key].lon]]


            # Add a PolyLine to the map to represent the flow between the nodes
            folium.PolyLine(points,
                            color=color,                                # Set the color of the line
                            weight=width,                               # Set line width
                            opacity=opacity,                            # Set line opacity
                            popup=var.X).add_to(map)                    # Show the flow value in a popup on the map


## **Data setup and preprocessing**

### **Nodes**

#### Reading input files

In [ ]:
# File containing customer data
customer_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/inventory_pooling_effect/customers.csv'

# Loading customer data into a pandas DataFrame
customers_df = pd.read_csv(customer_data_file)


# Displaying the first few rows of the DataFrame to verify the data
customers_df.head()

,ID,name,state,lat,lon,demand,sigma
0,c1,New York,New York,40.6943,-73.9249,189100,30256
1,c2,Los Angeles,California,34.1141,-118.4068,119200,23840
2,c3,Chicago,Illinois,41.8375,-87.6866,85000,15300
3,c4,Miami,Florida,25.7840,-80.2101,60800,18240
4,c5,Houston,Texas,29.7860,-95.3885,59700,16716


In [ ]:
# File containing warehouse data
warehouse_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/inventory_pooling_effect/warehouses.csv'

# Loading warehouse data into a pandas DataFrame
warehouses_df = pd.read_csv(warehouse_data_file)

# Displaying the first few rows of the DataFrame to verify the data
warehouses_df.head()


,ID,name,lat,lon,fixed_cost,capacity
0,w1,Dallas,32.776272,-96.796856,25000,4000000
1,w2,Columbus,39.962260,-83.000707,25000,2000000
2,w3,Atlanta,33.748992,-84.390264,25000,4000000
3,w4,Portland,45.520247,-122.674194,25000,6000000
4,w5,Savannah,32.056457,-81.095127,25000,4000000


#### Definition of Classes

In [ ]:
# Class representing a Customer object

class Customer():
    def __init__(self, ID, name, lat, lon, demand, sigma):
        self.ID = ID                    # Customer's ID
        self.name = name                # Customer's name
        self.lat = lat                  # Customer's latitude
        self.lon = lon                  # Customer's longitude
        self.demand = demand            # Customer's demand
        self.sigma = sigma              # Customer's standard deviation of demand

        self.active = True              # Initializing node as active


In [ ]:
# Class representing a Warehouse object

class Warehouse():
    def __init__(self, ID, name, lat, lon, fixed_cost, capacity):
        self.ID = ID                  # Warehouse's ID
        self.name = name              # Warehouse's name
        self.lat = lat                # Warehouse's latitude
        self.lon = lon                # Warehouse's longitude
        self.fixed_cost = fixed_cost  # Warehouse's fixed cost
        self.capacity = capacity      # Warehouse's fixed cost

        self.active = True            # Initializing node as active


#### Creating node objects

In [ ]:
# Initializing an empty dictionary to store node objects
nodes = dict()

In [ ]:
# Creating a dictionary of customer objects
customers = dict()
for i, row in customers_df.iterrows():
    customers[row['ID']] = Customer(ID=row['ID'],           # Customer's ID
                                    name=row['name'],       # Customer's name
                                    lat=row['lat'],         # Customer's latitude
                                    lon=row['lon'],         # Customer's longitude
                                    demand=row['demand'],   # Customer's demand
                                    sigma=row['sigma'],     # Customer's standard deviation of demand
                                    )

# Merging the customers dictionary into the existing nodes dictionary
nodes = {**nodes, **customers}

In [ ]:
# Creating a dictionary of warehouse objects
warehouses = dict()
for i, row in warehouses_df.iterrows():
    warehouses[row['ID']] = Warehouse(ID = row['ID'],               # Warehouse's ID
                                    name = row['name'],             # Warehouse's name
                                    lat = row['lat'],               # Warehouse's latitude
                                    lon = row['lon'],               # Warehouse's longitude
                                    fixed_cost = row['fixed_cost'], # Warehouse's fixed cost
                                    capacity = row['capacity'])     # Warehouse's fixed cost

# Merging the warehouse dictionary into the existing nodes dictionary
nodes = {**nodes, **warehouses}

#### Visualizing node objects

In [ ]:
# Create a new map centered on Europe with a zoom level of 5
map = folium.Map([40, -100.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', active_color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and yellow background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='yellow')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


### **Arcs**

#### Arc distances

In [ ]:
# Creating a dictionary containing distances between warehouses and customers

distances = dict()
for w, warehouse in warehouses.items():
  for c, customer in customers.items():
      distances[w, c] = compute_geodesic_distance(origin = warehouse, destination = customer, unit = 'km')

#### Arc costs

In [ ]:
cost_unit_km_warehouse_customer = 0.0005    # Cost per unit per kilometer from warehouse to customer

# Creating a dictionary containing unit costs between suppliers and warehouses, and between warehouses and customers
unit_cost = dict()

for w, warehouse in warehouses.items():                                                        # Iterate over warehouses
    for c, customer in customers.items():                                                      # Iterate over customers
        unit_cost[w, c] = distances[w, c] * cost_unit_km_warehouse_customer                    # Calculate unit cost as distance multiplied by cost per km (warehouse to customer)


### **Inventory parameters**

In [ ]:
# Additional inventory parameters for safety stock calculation

service_level = 0.95                                # Desired service level (probability of not running out of stock)
safety_stock_factor = st.norm.ppf(service_level)    # Safety stock factor based on the service level, using the inverse of the standard normal distribution

print('Safety Stock factor: ', round(safety_stock_factor, 3))


Safety Stock factor:  1.645


In [ ]:
# Additional inventory parameters for cost calculations

annual_carrying_charge = 0.2                        # Annual carrying charge as a percentage of the unit cost
unit_purchase_cost = 10                             # Purchase cost per unit of the product
ordering_cost = 10000                               # Fixed cost per order
lead_time = 60 / 365

## **Optimization model**

### **Creating and solving the optimization model**

In [ ]:
# Initialize the optimization model
model = grb.Model("Inventory Pooling")

# Define binary location variables for each warehouse
wh_location_vars = dict()
for w, warehouse in warehouses.items():
    wh_location_vars[w] = model.addVar(vtype=grb.GRB.BINARY,
                                       name="activation_wh{0}".format(w))

# Define allocation variables (binary) for each warehouse-customer pair
allocation_vars = dict()
for w, warehouse in warehouses.items():
    for c, customer in customers.items():
        allocation_vars[w, c] = model.addVar(vtype=grb.GRB.BINARY,
                                             name="allocation_cust_wh_{0}_{1}".format(w, c))

# Define auxiliary continuous variables
aux_var_u = dict()
aux_var_v = dict()
for w, warehouse in warehouses.items():
    aux_var_u[w] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                                name="aux_var_u{0}".format(w))
    aux_var_v[w] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                                name="aux_var_v{0}".format(w))


# Calculate fixed warehouse cost based on warehouse activation status
fixed_warehouse_cost = grb.quicksum(wh_location_vars[w]
                                    * warehouse.fixed_cost
                                    for w, warehouse in warehouses.items())

# Calculate ordering and holding costs
ordering_holding_cost = math.sqrt(2 *
                                  annual_carrying_charge *
                                  unit_purchase_cost * ordering_cost) * grb.quicksum(aux_var_u[w]
                                                                                     for w, warehouse in warehouses.items())

# Calculate safety stock cost
safety_stock_cost = (safety_stock_factor
                     * annual_carrying_charge
                     * unit_purchase_cost
                     * math.sqrt(lead_time)
                     * grb.quicksum(aux_var_v[w]
                                    for w, warehouse in warehouses.items()))

# Calculate transportation costs based on customer demands and allocation decisions
transportation_cost = grb.quicksum(unit_cost[w, c] * allocation_vars[w, c] * customer.demand
                                   for w, warehouse in warehouses.items()
                                   for c, customer in customers.items())

# Constraint: Each customer is assigned to exactly one warehouse
for c, customer in customers.items():
    model.addConstr(grb.quicksum(allocation_vars[w, c]
                                 for w, warehouse in warehouses.items()) == 1)

# Constraint: warehouse capacity constraint
for w, warehouse in warehouses.items():
  model.addConstr(grb.quicksum(allocation_vars[w, c] *
                               customer.demand
                               for c, customer in customers.items()) <= wh_location_vars[w] * warehouse.capacity)

# Constraint: Define aux_var_u as the square root of the demand allocated to each warehouse
for w, warehouse in warehouses.items():
    model.addConstr(aux_var_u[w]**2 == grb.quicksum(allocation_vars[w, c] * customer.demand
                                                    for c, customer in customers.items()))

# Constraint: Define aux_var_v based on customer demand variance allocated to each warehouse
for w, warehouse in warehouses.items():
    model.addConstr(aux_var_v[w]**2 == grb.quicksum(allocation_vars[w, c] * (customer.sigma**2)
                                                    for c, customer in customers.items()))

# Define the objective function as the total cost (fixed, ordering/holding, safety stock, and transportation)
total_cost = fixed_warehouse_cost + ordering_holding_cost + safety_stock_cost + transportation_cost
model.setObjective(total_cost, grb.GRB.MINIMIZE)

# Optimize the model to find the minimum cost solution
model.optimize()


# Update warehouse activation status based on optimization results
for w, warehouse in warehouses.items():
  warehouse.active = True if wh_location_vars[w].X == 1 else False



Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 41 rows, 195 columns and 365 nonzeros
Model fingerprint: 0x5fd3be3c
Model has 10 quadratic constraints
Variable types: 10 continuous, 185 integer (185 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+06]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+04, 9e+08]
  Objective range  [1e+00, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.00s
Presolved: 71 rows, 196 columns, 1465 nonzeros
Presolved model has 10 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 11 continuous, 185 integer (185 binary)
Found heuristic solution: objective 1490012.0608

R

### **Solution visualization and analysis**

In [ ]:
# Create a new map centered on Europe with a zoom level of 5
map = folium.Map([40, -100.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', active_color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and yellow background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='yellow')

# Plot flows on the map
plot_flows(map = map, vars = allocation_vars, nodes = nodes, opacity = 0.3)

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map
